# InfluxDB data loading

Writing orginal and preprocessed data into influxDB bu

In [1]:
from pyspark.sql import SparkSession
from influxdb_client import InfluxDBClient, Point
from influxdb_client.client.write_api import SYNCHRONOUS
import pandas as pd

In [2]:
# Spark setup
spark = SparkSession.builder.appName("mema").getOrCreate()
spark.sparkContext.setLogLevel("WARN")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/19 13:34:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/19 13:34:41 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:
def process_spark_dataframe(spark, csv_file_path, influxdb_bucket):
    # Read CSV file into a Spark DataFrame
    spark_df = spark.read.csv(csv_file_path, header=True, inferSchema=True)

    # InfluxDB setup
    influxdb_url = "http://localhost:8086"
    influxdb_token = "BD8-Z9Rcrb-lCOQcWJ7h-5kzuvX0ZIWMmlw8uza-1psB_8jjQOSbXC8XyOaWyjEBUWUZTXZtp-rYhnQPEZShxw=="
    influxdb_org = "mema_org"

    with InfluxDBClient(url=influxdb_url, token=influxdb_token, org=influxdb_org) as client:
        write_api = client.write_api(write_options=SYNCHRONOUS)

        # Iterate over rows in the Spark DataFrame
        for row in spark_df.collect():
            try:
                # Create InfluxDB Point
                point = Point("HAI_measurements")

                # Add fields to the InfluxDB Point, excluding "timestamp"
                for col_name in spark_df.columns:
                    if col_name != "timestamp":
                        col_value = row[col_name]
                        point.field(col_name, col_value)

                # Write the InfluxDB Point to the database
                write_api.write(influxdb_bucket, influxdb_org, point)

            except Exception as e:
                print(f"Error processing row: {e}")    

In [6]:
# Create a Spark session
# Set your CSV file path 
csv_test1 = "/Users/emmatosato/Documents/UNI_Locale/Erasmus/OST/ost-sm-change-detection/data_analysis/merged_data/test_pd1.csv"
csv_test2 = "/Users/emmatosato/Documents/UNI_Locale/Erasmus/OST/ost-sm-change-detection/data_analysis/merged_data/test_pd2.csv"
csv_complete = "/Users/emmatosato/Documents/UNI_Locale/Erasmus/OST/ost-sm-change-detection/data_analysis/merged_data/complete_pd.csv"

csv_test1_proc = "/Users/emmatosato/Documents/UNI_Locale/Erasmus/OST/ost-sm-change-detection/data_analysis/preprocessed_data/scaled_test1.csv"
csv_test2_proc = "/Users/emmatosato/Documents/UNI_Locale/Erasmus/OST/ost-sm-change-detection/data_analysis/preprocessed_data/scaled_test2.csv"
csv_complete_proc = "/Users/emmatosato/Documents/UNI_Locale/Erasmus/OST/ost-sm-change-detection/data_analysis/preprocessed_data/scaled_complete.csv"

In [ ]:
# Call the function
process_spark_dataframe(spark, csv_test1, "Test1")
process_spark_dataframe(spark, csv_test2, "Test2")

In [ ]:
# Call the function
process_spark_dataframe(spark, csv_test1_proc, "Test1Processed")
process_spark_dataframe(spark, csv_test2_proc, "Test2Processed")

23/12/18 21:28:28 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


23/12/19 01:02:27 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 6663154 ms exceeds timeout 120000 ms
23/12/19 01:02:28 WARN SparkContext: Killing executors is not supported by current scheduler.
23/12/19 02:16:37 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at 

In [ ]:
process_spark_dataframe(spark, csv_complete, "CompleteTrain")
process_spark_dataframe(spark, csv_complete_proc, "CompleteTrainProcessed")